# ANN with PyTorch

* Model:simple neural net
* Optimiser: SGD with mini batches
* Metric: accuracy
* Loss: cross entropy

Source: [Pytorch tutorial](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)

In [0]:
# Hyperparameters
batch_size = 64 # for dataloader
learning_rate = 1e-3
num_epochs = 5

# Model definition
input_size = 28*28
num_hidden_activations = 512
num_classes = 10

In [0]:
!pip3 install torch

In [0]:
!pip install fastai

In [0]:
from fastai.vision.all import *
import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F

In [0]:
set_seed(2*33) # Set random seed for random, torch, and numpy (where available)

## Load data

In [0]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path # important for torchvision.datasets.ImageFolder

In [0]:
transform = transforms.Compose(
    [transforms.Grayscale(), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)

In [0]:
full_dataset = torchvision.datasets.ImageFolder((path/"train").as_posix(), transform = transform)

# Splitting the above dataset into a training and validation dataset
train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
training_set, validation_set = torch.utils.data.random_split(full_dataset, [train_size, valid_size])

In [0]:
train_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size)

## Define model

In [0]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [0]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden_activations, num_classes):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, num_hidden_activations),
            nn.ReLU(),
            nn.Linear(num_hidden_activations, num_hidden_activations),
            nn.ReLU(),
            nn.Linear(num_hidden_activations, num_classes),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [0]:
model = NeuralNetwork(input_size, num_hidden_activations, num_classes).to(device)

In [0]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

## Optimise params = train model

In [0]:
for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, criterion, optimizer)
    test_loop(validation_loader, model, criterion)
print("Done!")